In [153]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

In [154]:
big_countries = ["AUS", "BRA", "CAN", "CHN", "IND", "RUS", "USA"]

yrs = range(1980, 2016+1)
yrs_s = [str(xx) for xx in yrs]

In [155]:
dr_states = pd.read_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/input_damage_distribution/natcat_drought_states_AUS-BRA-CAN-CHN-IND-RUS-USA.csv', index_col="event_id")
dr_states.head(50)

,iso3,Affected Area (Detail),affected_states,affected_states_dose
event_id,,,,
419,AUS,New South Wales,New South Wales (New South Wales),AUS.5_1
1102,AUS,"Victoria; New South Wales, Broken Hill; Queens...",Victoria (Victoria); New South Wales (New Sout...,"AUS.10_1, AUS.5_1, AUS.7_1, AUS.8_1"
1251,AUS,"South Australia; Victoria; New South Wales, Br...",South Australia (South Australia); Victoria (V...,"AUS.10_1, AUS.5_1, AUS.7_1, AUS.8_1"
5266,AUS,Queensland; South Australia,Queensland (Queensland); South Australia (Sout...,"AUS.7_1, AUS.8_1"
6406,AUS,Queensland; Western Australia; South Australia...,Queensland (Queensland); Western Australia (We...,"AUS.11_1, AUS.5_1, AUS.7_1, AUS.8_1"
8631,AUS,Queensland; New South Wales,Queensland (Queensland); New South Wales (New ...,"AUS.5_1, AUS.7_1"
9536,AUS,"New South Wales, Mo naro area; Queensland; Vic...","New South Wales (New South Wales), Mo naro are...","AUS.10_1, AUS.5_1, AUS.7_1"
9824,AUS,New South Wales,New South Wales (New South Wales),AUS.5_1
10135,AUS,"New South Wales, Shoalhaven; Northern Territor...","New South Wales (New South Wales), Shoalhaven ...","AUS.5_1, AUS.6_1, AUS.7_1"


In [156]:
dose = pd.read_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/input_damage_distribution/DoseRegions.csv')

In [157]:
tot_gdp = pd.read_csv("/home/insauer/projects/droughts/droughts/data/socio_economic/input_damage_distribution/DOSE_V2_grp_usd_2015_running_11yr_mean.csv", index_col=0)

In [158]:
natcat_damage = pd.read_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/damage_converted/natcat_droughts_converted.csv', index_col=0)
natcat_damage

,year,begin,begin_yr,begin_mon,end,end_yr,end_mon,event,type,subtype,...,Association Type \n(Single + Main = Single-Country-Event; Country = Country data from Multi-Country-Event; Region =Multi-Country-Event; see sheet 'Read me'),Country Name,Affected Area,Affected Area (Detail),Description,Catastrophe Class,Geocoded Place,damage_constant2015USD_PPP,damage_constant2017USD_PPP,conversion_source
id,,,,,,,,,,,,,,,,,,,,,
0,1980,01/01/1980 00:00,1980,1,31/08/1982 00:00,1982,8,Drought,D,dr:Drought,...,Country,Spain,"S, C",Murcia; Andalucia; Baleares,"Lack of rain, total abstinence of rain, dry sp...",4,ESP: Murcia,4824.336569,5166.227764,GDPuc_currentUSD_to_constantXPPP
9,1980,01/01/1980 00:00,1980,1,31/01/1980 00:00,1980,1,Drought,D,dr:Drought,...,Single,Somalia,"Entire country,",n.a,"Lack of rain. Cropland damaged, livestock affe...",3,SOM:,0.069383,0.070457,GDPuc_currentUSD_to_constantXPPP
10,1980,01/01/1980 00:00,1980,1,31/12/1980 00:00,1980,12,Drought,D,dr:Drought,...,Country,Mozambique,"S, C,",Manica; Sofala; Inhambane; Gaza,"Lack of rain, dry spell, hot temperatures. Riv...",3,MOZ: Manica,193.657903,187.422649,GDPuc_constant2016MER_to_constantXPPP
11,1980,01/01/1980 00:00,1980,1,31/12/1985 00:00,1985,12,Drought,D,dr:Drought,...,Country,Gambia,"Entire country,",n.a,"Lack of rain, dry spell. Cropland damaged/dest...",3,GMB:,66.349153,66.085882,GDPuc_currentUSD_to_constantXPPP
12,1980,01/01/1980 00:00,1980,1,31/12/1982 00:00,1982,12,Drought,D,dr:Drought,...,Country,Mali,"W,","Kayes, Timbuktu","Lack of rain, dry spell, hot temperatures. Cro...",3,MLI: Kayes,125.630406,132.248623,GDPuc_currentUSD_to_constantXPPP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29778,2016,01/07/2016 00:00,2016,7,17/08/2016 00:00,2016,8,Drought,D,dr:Drought,...,Single,France,"W, C, S,","Eure-et-Loir, Indre, Loire-Atlantique, Vendée,...","Dry conditions, lack of rain. Water restrictio...",0,"FRA: Loiré, Loiré",0.111133,0.117923,GDPuc_currentUSD_to_constantXPPP
29936,2016,01/08/2016 00:00,2016,8,05/09/2016 00:00,2016,9,Drought,D,dr:Drought,...,Single,China,"N,",Gansu,"Lack of rain. Crops (esp. corn, potato) destro...",3,CHN: Gansu Sheng,761.754029,744.912423,GDPuc_currentUSD_to_constantXPPP
30097,2016,01/09/2016 00:00,2016,9,24/09/2016 00:00,2016,9,Drought,D,dr:Drought,...,Single,China,"C,",Hubei,"Lack of rain. Shortage of drinking water. >1,2...",2,CHN: Hubei,101.567204,99.321656,GDPuc_currentUSD_to_constantXPPP


In [159]:
natcat_damage['damage_constant2015USD_PPP'] = natcat_damage['damage_constant2015USD_PPP'] * 1e6

In [160]:
dr_states.loc[21813, "affected_states_dose"].split(", ")

['BRA.11_1', 'BRA.12_1', 'BRA.4_1']

In [161]:

natcat_new = pd.DataFrame(columns=[
        "event_id", "yrmon_begin", "yrmon_end", "subtype", "damage_constant2015USD_PPP", "iso3", "state", "affected",
        "affected_area", "affected_states", 'total_gdp', 'total_gdp_weight', 'total_gdp_damage'])
for ev in natcat_damage.index:
    print(ev)
    yr_begin, mon_begin, yr_end, mon_end = natcat_damage.loc[ev, ["begin_yr", "begin_mon", "end_yr", "end_mon"]]
    yrmon_ev = [f"{yr_begin}{str(mon_begin).zfill(2)}", f"{yr_end}{str(mon_end).zfill(2)}"]
    data_ev = pd.DataFrame(index=[0], columns=natcat_new.columns)
        # do not distribute the damages, just take the national damage vs. the national indicator.
    iso3 = natcat_damage.loc[ev, "iso3"]    
    data_ev.loc[:, "event_id"] = ev
    data_ev.loc[:, ["yrmon_begin", "yrmon_end"]] = yrmon_ev
    data_ev.loc[:, ["subtype", "damage_constant2015USD_PPP", "iso3"]] = natcat_damage.loc[ev, ["subtype", "damage_constant2015USD_PPP", "iso3"]].values
    data_ev.loc[:, "state"] = iso3
    data_ev.loc[:, "affected"] = True
    data_ev.loc[:, "affected_area"] = "national"
    data_ev.loc[:, "affected_states"] = "national"
    data_ev.loc[:, "total_gdp_weight"] = 1
    data_ev.loc[:, "total_gdp_damage"] = natcat_damage.loc[ev, "damage_constant2015USD_PPP"]
    natcat_new = pd.concat([natcat_new, data_ev], ignore_index=True)
    if natcat_damage.loc[ev, "iso3"] in big_countries:
        states_aff = dr_states.loc[ev, "affected_states_dose"].split(", ") # affected states, dose codes.
        codes_iso = dose.GID_1[dose.GID_0 == iso3].tolist()
        states_naff = [xx for xx in codes_iso if xx not in states_aff]
        data_ev = pd.DataFrame(index=codes_iso, columns=natcat_new.columns)
        data_ev.loc[:, "state"] = codes_iso
        data_ev.loc[:, "event_id"] = ev
        data_ev.loc[:, ["yrmon_begin", "yrmon_end"]] = yrmon_ev
        data_ev.loc[:, ["subtype", "damage_constant2015USD_PPP", "iso3"]] = natcat_damage.loc[ev, ["subtype", "damage_constant2015USD_PPP", "iso3"]].values
        data_ev.loc[:, "affected_area"] = "states"
        data_ev.loc[:, "affected_states"] = ", ".join(states_aff)
        data_ev.loc[states_aff, "affected"] = True
        data_ev.loc[states_naff, "affected"] = False
        
        ev_yrs = [str(xx) for xx in range(yr_begin, yr_end+1)]
        # there might be some missing data -> this state is set to weight = 0.
        gdp_states = tot_gdp.loc[:, ev_yrs].mean(axis=1).reindex(index=states_aff + states_naff)
        data_ev.loc[:, 'total_gdp'] = gdp_states
        weights = gdp_states.loc[states_aff].div(gdp_states.loc[states_aff].sum())
        # set weights 0 if they are NaN.
        weights[weights.isnull()] = 0
        # if there is only one weight -> it is 1.
        if len(weights) == 1:
            weights[:] = 1 # is not calculated as 1 if it is NaN.
        
        data_ev.loc[states_aff, "total_gdp_weight"] = weights.values
        data_ev.loc[:, "total_gdp_damage"] = data_ev.loc[:, "total_gdp_weight"].multiply(natcat_damage.loc[ev, "damage_constant2015USD_PPP"])
        
        if weights.sum() == 0:
            print("\nfor", iso3, "tot_gdp", "and event", ev, "the weights are all 0, even though a state was affected")
            print("  len(weights) =", len(weights))
            print("  affected states:", states_aff)
            print(data_ev.loc[states_aff, ["state", "total_gdp", "total_gdp_weight", "total_gdp_damage"]])
        natcat_new = pd.concat([natcat_new, data_ev], ignore_index=True)

0
9
10
11
12
13
14
15
16
81
158
159
160
161
162
163
166
168
329
360
418
419
420
421
422
455
456
457
581
617
665
702
751
824
825
826
889
928
931
982
1019
1020
1024
1062
1102
1103
1145
1175
1176
1251
1255
1256
1257
1258
1303
1304
1305
1306
1309
1310
1378
1380
1382
1383
1419
1421
1422
1423
1452
1453
1454
1488
1620
1621
1648
1680
1681
1711
1716
1717
1718
1719
1752
1860
1923
1977
2042
2055
2057
2238
2572
2604
2608
2609
2648
2681
2879
2895
2896
2897
2898
2899
2900
2901
2902
2984
3023
3025
3053
3093
3094
3150
3151
3278
3316
3423
3451
3452
3453
3454
3457
3458
3459
3460
3461
3462
3555
3557
3558
3594
3601
3622
3632
3674
3786
3787
3855
3858
3914
3915
4010
4011
4047
4059
4061
4096
4141
4175
4211
4316
4317
4389
4539
4540
4567
4604
4614
4615
4616
4667
4808
4809
4855
4919
4983
4984
5040
5137
5189
5221
5225
5226
5265
5266
5267
5268
5270
5309
5334
5335
5336
5340
5341
5371
5372
5463
5513
5518
5601
5689
5777
5778
5826
5835
5841
5923
5971
6005
6007
6052
6053
6054
6123
6190
6191
6258
6351
6406
6446
6453
65

In [162]:
natcat_new.to_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/damage_distributed/natcat_converted_distributed.csv')

In [163]:
natcat_new.loc[natcat_new['event_id']==13813]

,event_id,yrmon_begin,yrmon_end,subtype,damage_constant2015USD_PPP,iso3,state,affected,affected_area,affected_states,total_gdp,total_gdp_weight,total_gdp_damage
2927,13813,200203,200301,dr:Drought,4006051179.90124,AUS,AUS,True,national,national,NaN,1,4006051179.90124
2928,13813,200203,200301,dr:Drought,4006051179.90124,AUS,AUS.10_1,True,states,"AUS.10_1, AUS.11_1, AUS.1_1, AUS.2_1, AUS.3_1,...",168260818195.574402,0.243404,975088156.882142
2929,13813,200203,200301,dr:Drought,4006051179.90124,AUS,AUS.11_1,True,states,"AUS.10_1, AUS.11_1, AUS.1_1, AUS.2_1, AUS.3_1,...",78041894623.369171,0.112894,452260532.213987
2930,13813,200203,200301,dr:Drought,4006051179.90124,AUS,AUS.1_1,True,states,"AUS.10_1, AUS.11_1, AUS.1_1, AUS.2_1, AUS.3_1,...",NaN,0.0,0.0
2931,13813,200203,200301,dr:Drought,4006051179.90124,AUS,AUS.2_1,True,states,"AUS.10_1, AUS.11_1, AUS.1_1, AUS.2_1, AUS.3_1,...",14179578806.014971,0.020512,82172067.814692
2932,13813,200203,200301,dr:Drought,4006051179.90124,AUS,AUS.3_1,True,states,"AUS.10_1, AUS.11_1, AUS.1_1, AUS.2_1, AUS.3_1,...",NaN,0.0,0.0
2933,13813,200203,200301,dr:Drought,4006051179.90124,AUS,AUS.4_1,True,states,"AUS.10_1, AUS.11_1, AUS.1_1, AUS.2_1, AUS.3_1,...",NaN,0.0,0.0
2934,13813,200203,200301,dr:Drought,4006051179.90124,AUS,AUS.5_1,True,states,"AUS.10_1, AUS.11_1, AUS.1_1, AUS.2_1, AUS.3_1,...",238951552500.443176,0.345664,1384747984.76828
2935,13813,200203,200301,dr:Drought,4006051179.90124,AUS,AUS.6_1,True,states,"AUS.10_1, AUS.11_1, AUS.1_1, AUS.2_1, AUS.3_1,...",8055388159.657495,0.011653,46681774.627061
2936,13813,200203,200301,dr:Drought,4006051179.90124,AUS,AUS.7_1,True,states,"AUS.10_1, AUS.11_1, AUS.1_1, AUS.2_1, AUS.3_1,...",123836186210.44249,0.17914,717643001.277348


In [164]:
yrmons = pd.Series([int(f"{yr}{str(mon).zfill(2)}") for yr in yrs for mon in range(1, 12+1)], dtype=int)
regions = sorted(natcat_new.state.unique())
dams_ts = pd.DataFrame(0.0, index=regions, columns=yrs)
for row in natcat_new.index:
        
    data_act = natcat_new.loc[row, :]
    # damages distributed to years by equally weighting with event months.
    # put the damage div. number of event months into a pd.Series with the event yrmons as index.
    ev_dams = pd.Series(index=yrmons[(yrmons>=int(data_act.yrmon_begin)) & (yrmons<=int(data_act.yrmon_end))], dtype=float)
    ev_dams[:] = data_act["total_gdp_damage"]/len(ev_dams)
    for yr in sorted(set([str(xx)[:4] for xx in ev_dams.index])): # each event year, as string.
        print(yr)
        dams_ts.loc[data_act.state, int(yr)] = dams_ts.loc[data_act.state, int(yr)] + \
            ev_dams.loc[[xx for xx in ev_dams.index if str(xx)[:4] == yr]].sum()

1980
1981
1982
1980
1980
1980
1981
1982
1983
1984
1985
1980
1981
1982
1980
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1980
1981
1982
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1981
1982
1980
1981
1982
1983
1984
1985
1980
1981
1982
1983
1980
1981


In [165]:
yrmons = pd.Series([int(f"{yr}{str(mon).zfill(2)}") for yr in yrs for mon in range(1, 12+1)], dtype=int)
regions = sorted(natcat_new.state.unique())
dams_ts_bin = pd.DataFrame(0.0, index=regions, columns=yrs)
for row in natcat_new.index:
        
    data_act = natcat_new.loc[row, :]
    # damages distributed to years by equally weighting with event months.
    # put the damage div. number of event months into a pd.Series with the event yrmons as index.
    yr_begin=int(data_act.yrmon_begin[:-2])
    yr_end=int(data_act.yrmon_end[:-2])
    ev_dams =np.arange(yr_begin, yr_end+1)
    for yr in ev_dams: # each event year, as string.
        print(yr)
        dams_ts_bin.loc[data_act.state, int(yr)] = 1

1980
1981
1982
1980
1980
1980
1981
1982
1983
1984
1985
1980
1981
1982
1980
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1982
1983
1980
1980
1981
1982
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1980
1981
1982
1980
1981
1982
1983
1984
1985
1980
1981
1982
1983
1980
1981


In [166]:
dams_ts_bin

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
AFG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
AGO,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
ALB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ARG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
ARM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VUT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
WSM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
ZAF,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
ZMB,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [167]:
dams_ts

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
AFG,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.501701e+08,3.751418e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
AGO,0.000000e+00,1.857970e+08,0.000000e+00,0.000000e+00,0.000000e+00,4.768621e+08,0.000000e+00,0.0,0.000000e+00,1.204557e+08,...,0.000000e+00,8.797946e+06,0.000000e+00,0.000000e+00,0.000000e+00,8.632569e+06,5.844557e+07,1.461139e+07,5.426166e+08,8.500125e+07
ALB,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,2.522113e+08,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
ARG,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,6.069764e+07,0.000000e+00,...,4.740772e+08,4.740772e+08,4.740772e+08,1.185193e+08,2.578278e+09,1.289139e+09,4.705648e+08,0.000000e+00,0.000000e+00,0.000000e+00
ARM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VUT,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.050820e+04,0.000000e+00
WSM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.394648e+04,0.000000e+00
ZAF,2.146065e+07,0.000000e+00,3.720068e+09,3.720068e+08,0.000000e+00,0.000000e+00,1.007422e+08,0.0,2.100379e+07,0.000000e+00,...,5.569649e+07,3.037990e+07,0.000000e+00,8.172339e+06,0.000000e+00,0.000000e+00,7.369260e+06,1.842315e+06,3.064411e+09,1.180437e+08
ZMB,0.000000e+00,0.000000e+00,1.333077e+08,1.333077e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.563269e+07,2.697239e+08


In [168]:
dams_ts.to_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/damage_distributed/natcat_annual_converted_distributed.csv')

In [169]:
dams_ts_bin.to_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/damage_distributed/natcat_annual_converted_distributed_binary.csv')

# EM-DAT 

In [170]:
emdat_damage=pd.read_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/damage_converted/emdat_droughts_converted.csv')
emdat_damage

,Unnamed: 0,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,...,"Insured Damage, Adjusted ('000 US$)",damage_currentUSD_MER,damage_constant2024USD_MER,CPI,Admin Units,Entry Date,Last Update,damage_constant2015USD_PPP,damage_constant2017USD_PPP,conversion_source
0,1213,1980-9005-DJI,Yes,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,27.045581,NaN,2006-02-13,2023-09-25,NaN,NaN,GDPuc_currentUSD_to_constantXPPP
1,1214,1980-9005-SDN,Yes,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,27.045581,NaN,2006-02-13,2023-09-25,NaN,NaN,GDPuc_currentUSD_to_constantXPPP
2,1215,1980-9005-SOM,Yes,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,27.045581,NaN,2006-02-13,2023-09-25,NaN,NaN,GDPuc_currentUSD_to_constantXPPP
3,1216,1980-9011-LKA,Yes,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,27.045581,NaN,2006-02-13,2023-09-25,NaN,NaN,GDPuc_currentUSD_to_constantXPPP
4,1217,1980-9114-BEN,Yes,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,651.0,1992.0,32.687705,NaN,2006-09-11,2023-09-25,4932.701672,5033.755256,GDPuc_currentUSD_to_constantXPPP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,13042,2024-9075-COL,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,GLIDE:DR-2024-000013,NaN,...,NaN,NaN,NaN,NaN,"[{""adm1_code"":939,""adm1_name"":""Boyaca""},{""adm1...",2024-02-13,2024-04-10,NaN,NaN,GDPuc_currentUSD_to_constantXPPP
667,13043,2024-9131-ZMB,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,GLIDE:DR-2024-000018,NaN,...,NaN,NaN,NaN,NaN,NaN,2024-03-12,2024-06-06,NaN,NaN,GDPuc_currentUSD_to_constantXPPP
668,13044,2024-9135-MHL,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,"[{""adm1_code"":2002,""adm1_name"":""Administrative...",2024-03-13,2024-04-10,NaN,NaN,GDPuc_currentUSD_to_constantXPPP
669,13045,2024-9197-MWI,No,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2024-04-05,2024-05-14,NaN,NaN,GDPuc_currentUSD_to_constantXPPP


In [171]:
emdat_damage=emdat_damage.rename(columns={'Unnamed: 0': 'event_id'})
emdat_damage.index=emdat_damage.event_id

In [172]:
emdat_damage.columns

Index(['event_id', 'DisNo.', 'Historic', 'Classification Key',
       'Disaster Group', 'Disaster Subgroup', 'Disaster Type',
       'Disaster Subtype', 'External IDs', 'Event Name', 'ISO', 'Country',
       'Subregion', 'Region', 'Location', 'Origin', 'Associated Types',
       'OFDA/BHA Response', 'Appeal', 'Declaration',
       'AID Contribution ('000 US$)', 'Magnitude', 'Magnitude Scale',
       'Latitude', 'Longitude', 'River Basin', 'Start Year', 'Start Month',
       'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths',
       'No. Injured', 'No. Affected', 'No. Homeless', 'Total Affected',
       'Reconstruction Costs ('000 US$)',
       'Reconstruction Costs, Adjusted ('000 US$)',
       'Insured Damage ('000 US$)', 'Insured Damage, Adjusted ('000 US$)',
       'damage_currentUSD_MER', 'damage_constant2024USD_MER', 'CPI',
       'Admin Units', 'Entry Date', 'Last Update',
       'damage_constant2015USD_PPP', 'damage_constant2017USD_PPP',
       'conversion_source'],

In [173]:
emdat_dr_states=pd.read_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/input_damage_distribution/emdat_drought_states_AUS-BRA-CAN-CHN-IND-RUS-USA.csv')

In [174]:
emdat_dr_states=emdat_dr_states.rename(columns={'Unnamed: 0': 'event_id'})

In [175]:
emdat_dr_states.index=emdat_dr_states.event_id

In [176]:
natcat_damage

,year,begin,begin_yr,begin_mon,end,end_yr,end_mon,event,type,subtype,...,Association Type \n(Single + Main = Single-Country-Event; Country = Country data from Multi-Country-Event; Region =Multi-Country-Event; see sheet 'Read me'),Country Name,Affected Area,Affected Area (Detail),Description,Catastrophe Class,Geocoded Place,damage_constant2015USD_PPP,damage_constant2017USD_PPP,conversion_source
id,,,,,,,,,,,,,,,,,,,,,
0,1980,01/01/1980 00:00,1980,1,31/08/1982 00:00,1982,8,Drought,D,dr:Drought,...,Country,Spain,"S, C",Murcia; Andalucia; Baleares,"Lack of rain, total abstinence of rain, dry sp...",4,ESP: Murcia,4.824337e+09,5166.227764,GDPuc_currentUSD_to_constantXPPP
9,1980,01/01/1980 00:00,1980,1,31/01/1980 00:00,1980,1,Drought,D,dr:Drought,...,Single,Somalia,"Entire country,",n.a,"Lack of rain. Cropland damaged, livestock affe...",3,SOM:,6.938290e+04,0.070457,GDPuc_currentUSD_to_constantXPPP
10,1980,01/01/1980 00:00,1980,1,31/12/1980 00:00,1980,12,Drought,D,dr:Drought,...,Country,Mozambique,"S, C,",Manica; Sofala; Inhambane; Gaza,"Lack of rain, dry spell, hot temperatures. Riv...",3,MOZ: Manica,1.936579e+08,187.422649,GDPuc_constant2016MER_to_constantXPPP
11,1980,01/01/1980 00:00,1980,1,31/12/1985 00:00,1985,12,Drought,D,dr:Drought,...,Country,Gambia,"Entire country,",n.a,"Lack of rain, dry spell. Cropland damaged/dest...",3,GMB:,6.634915e+07,66.085882,GDPuc_currentUSD_to_constantXPPP
12,1980,01/01/1980 00:00,1980,1,31/12/1982 00:00,1982,12,Drought,D,dr:Drought,...,Country,Mali,"W,","Kayes, Timbuktu","Lack of rain, dry spell, hot temperatures. Cro...",3,MLI: Kayes,1.256304e+08,132.248623,GDPuc_currentUSD_to_constantXPPP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29778,2016,01/07/2016 00:00,2016,7,17/08/2016 00:00,2016,8,Drought,D,dr:Drought,...,Single,France,"W, C, S,","Eure-et-Loir, Indre, Loire-Atlantique, Vendée,...","Dry conditions, lack of rain. Water restrictio...",0,"FRA: Loiré, Loiré",1.111329e+05,0.117923,GDPuc_currentUSD_to_constantXPPP
29936,2016,01/08/2016 00:00,2016,8,05/09/2016 00:00,2016,9,Drought,D,dr:Drought,...,Single,China,"N,",Gansu,"Lack of rain. Crops (esp. corn, potato) destro...",3,CHN: Gansu Sheng,7.617540e+08,744.912423,GDPuc_currentUSD_to_constantXPPP
30097,2016,01/09/2016 00:00,2016,9,24/09/2016 00:00,2016,9,Drought,D,dr:Drought,...,Single,China,"C,",Hubei,"Lack of rain. Shortage of drinking water. >1,2...",2,CHN: Hubei,1.015672e+08,99.321656,GDPuc_currentUSD_to_constantXPPP


In [177]:
yrs = range(1980, 2021)
yrs_s = [str(xx) for xx in yrs]

In [178]:
emdat_new = pd.DataFrame(columns=[
        "event_id", "yrmon_begin", "yrmon_end", "subtype", "damage_constant2015USD_PPP", "iso3", "state", "affected",
        "affected_area", "affected_states", 'total_gdp', 'total_gdp_weight', 'total_gdp_damage'])
for ev in emdat_damage.index:
    print(ev)
    yr_begin, mon_begin, yr_end, mon_end = emdat_damage.loc[ev, ["Start Year", "Start Month", "End Year", "End Month"]].replace([np.nan], [14]).astype(int)
    if mon_end==14:
        if yr_begin != yr_end:
            mon_end=6
            if mon_begin==14:
                mon_begin=6
        else:
            if mon_begin==14:
                mon_begin =1
                mon_end=12
            else:
                mon_end=12
            
        
    if yr_begin>2020:
        continue
    yrmon_ev = [f"{int(yr_begin)}{str(int(mon_begin)).zfill(2)}", f"{int(yr_end)}{str(int(mon_end)).zfill(2)}"]
    data_ev = pd.DataFrame(index=[0], columns=emdat_new.columns)
        # do not distribute the damages, just take the national damage vs. the national indicator.
    iso3 = emdat_damage.loc[ev, "ISO"]    
    data_ev.loc[:, "event_id"] = ev
    data_ev.loc[:, ["yrmon_begin", "yrmon_end"]] = yrmon_ev
    data_ev.loc[:, ["subtype", "damage_constant2015USD_PPP", "iso3"]] = emdat_damage.loc[ev, ["Disaster Subtype", "damage_constant2015USD_PPP", "ISO"]].values
    data_ev.loc[:, "state"] = iso3
    data_ev.loc[:, "affected"] = True
    data_ev.loc[:, "affected_area"] = "national"
    data_ev.loc[:, "affected_states"] = "national"
    data_ev.loc[:, "total_gdp_weight"] = 1
    data_ev.loc[:, "total_gdp_damage"] = emdat_damage.loc[ev, "damage_constant2015USD_PPP"]
    emdat_new = pd.concat([emdat_new, data_ev], ignore_index=True)
    if emdat_damage.loc[ev, "ISO"] in big_countries:
        states_aff = emdat_dr_states.loc[ev, "affected_area"].split(", ") # affected states, dose codes.
        codes_iso = dose.GID_1[dose.GID_0 == iso3].tolist()
        states_naff = [xx for xx in codes_iso if xx not in states_aff]
        
        data_ev = pd.DataFrame(index=codes_iso, columns=emdat_new.columns)
        data_ev.loc[:, "state"] = codes_iso
        data_ev.loc[:, "event_id"] = ev
        data_ev.loc[:, ["yrmon_begin", "yrmon_end"]] = yrmon_ev
        data_ev.loc[:, ["subtype", "damage_constant2015USD_PPP", "iso3"]] = emdat_damage.loc[ev, ["Disaster Subtype", "damage_constant2015USD_PPP", "ISO"]].values
        data_ev.loc[:, "affected_area"] = "states"
        data_ev.loc[:, "affected_states"] = ", ".join(states_aff)
        data_ev.loc[states_aff, "affected"] = True
        data_ev.loc[states_naff, "affected"] = False
        
        ev_yrs = [str(xx) for xx in range(int(yr_begin), int(yr_end)+1)]
        # there might be some missing data -> this state is set to weight = 0.
        gdp_states = tot_gdp.loc[:, ev_yrs].mean(axis=1).reindex(index=states_aff + states_naff)
        data_ev.loc[:, 'tot_gdp'] = gdp_states
        weights = gdp_states.loc[states_aff].div(gdp_states.loc[states_aff].sum())
        # set weights 0 if they are NaN.
        weights[weights.isnull()] = 0
        # if there is only one weight -> it is 1.
        if len(weights) == 1:
            weights[:] = 1 # is not calculated as 1 if it is NaN.
        
        data_ev.loc[states_aff, "tot_gdp_weight"] = weights.values
        data_ev.loc[:, "tot_gdp_damage"] = data_ev.loc[:, "tot_gdp_weight"].multiply(emdat_damage.loc[ev, "damage_constant2015USD_PPP"])
        
        if weights.sum() == 0:
            print("\nfor", iso3, "tot_gdp", "and event", ev, "the weights are all 0, even though a state was affected")
            print("  len(weights) =", len(weights))
            print("  affected states:", states_aff)
            print(data_ev.loc[states_aff, ["state", "tot_gdp", "tot_gdp_weight", "tot_gdp_damage"]])
        emdat_new = pd.concat([emdat_new, data_ev], ignore_index=True)

1213
1214
1215
1216
1217
1218
1219
1220
1221
1222
1223
1224
1225
1226
1227
1228
1229
1230
1231
1232
1233
1234
1235
1236
1337
1338
1339
1340
1341
1342
1343
1344
1345
1346
1347
1348
1349
1350
1351
1352
1353
1354
1471
1472
1473
1474
1475
1476
1609
1610
1611
1612
1613
1614
1615
1616


/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  yr_begin, mon_begin, yr_end, mon_end = emdat_damage.loc[ev, ["Start Year", "Start Month", "End Year", "End Month"]].replace([np.nan], [14]).astype(int)
/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  yr_begin, mon_begin, yr_end, mon_end = emdat_damage.loc[ev, ["Start Year", "Start Month", "End Year", "End Month"]].replace([np.nan], [14]).astype(int)
/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting

1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1748
1749
1750
1751
1752
1753
1886
1887
1888
2007
2008
2009
2010
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2331
2332
2333
2334
2335
2336
2337
2338
2339
2340
2341
2342
2343
2344
2345
2346
2347
2488
2489
2490
2491
2492
2493
2494
2712
2713
2714
2715
2716
2717
2718
2719
2720
2721
2722
2723


/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  yr_begin, mon_begin, yr_end, mon_end = emdat_damage.loc[ev, ["Start Year", "Start Month", "End Year", "End Month"]].replace([np.nan], [14]).astype(int)
/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  yr_begin, mon_begin, yr_end, mon_end = emdat_damage.loc[ev, ["Start Year", "Start Month", "End Year", "End Month"]].replace([np.nan], [14]).astype(int)
/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting

2724
2725
2726
2727
2728
2729
2730
2731
2898
2899
2900
2901
2902
2903
2904
2905
2906
2907
2908
2909
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3299
3300
3301
3302
3303
3304


...

/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  yr_begin, mon_begin, yr_end, mon_end = emdat_damage.loc[ev, ["Start Year", "Start Month", "End Year", "End Month"]].replace([np.nan], [14]).astype(int)
/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  yr_begin, mon_begin, yr_end, mon_end = emdat_damage.loc[ev, ["Start Year", "Start Month", "End Year", "End Month"]].replace([np.nan], [14]).astype(int)
/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting

10441
10749
10750
10751
10752
10753
10754
10755
10756
10757
10758
10759
10760
11034
11035
11036
11037
11038
11039
11040
11041
11042
11043
11044
11045
11046
11047
11048
11049
11050
11397
11398
11399
11400
11401
11402


/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  yr_begin, mon_begin, yr_end, mon_end = emdat_damage.loc[ev, ["Start Year", "Start Month", "End Year", "End Month"]].replace([np.nan], [14]).astype(int)
/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  yr_begin, mon_begin, yr_end, mon_end = emdat_damage.loc[ev, ["Start Year", "Start Month", "End Year", "End Month"]].replace([np.nan], [14]).astype(int)
/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting

11403
11404
11405
11406
11407
11408
11409
11410
11411
11774
11775
11776
11777
11778
11779
11780
11781
11782
11783
11784
11785
12163
12164
12165
12166
12167
12168
12169
12170
12171
12172
12173
12174
12175
12176
12177
12178
12539
12540
12541
12542
12543
12544
12545
12546
12547
12548
12549
12550
12551
12552
12553
12554
12555
12556
12557
12558
12559
12560
12561
12915
12916
12917
12918
12919
12920
12921
12922
12923
12924
13042
13043
13044
13045
13046


/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  yr_begin, mon_begin, yr_end, mon_end = emdat_damage.loc[ev, ["Start Year", "Start Month", "End Year", "End Month"]].replace([np.nan], [14]).astype(int)
/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  yr_begin, mon_begin, yr_end, mon_end = emdat_damage.loc[ev, ["Start Year", "Start Month", "End Year", "End Month"]].replace([np.nan], [14]).astype(int)
/tmp/ipykernel_34483/2175524961.py:6: FutureWarning: Downcasting

In [179]:
emdat_new.loc[emdat_new['event_id']==1887]

,event_id,yrmon_begin,yrmon_end,subtype,damage_constant2015USD_PPP,iso3,state,affected,affected_area,affected_states,total_gdp,total_gdp_weight,total_gdp_damage,tot_gdp,tot_gdp_weight,tot_gdp_damage
226,1887,198506,198606,Drought,NaN,CHN,CHN,True,national,national,NaN,1,NaN,NaN,NaN,NaN
227,1887,198506,198606,Drought,NaN,CHN,CHN.10_1,False,states,"CHN.16_1, CHN.31_1, CHN.4_1, CHN.6_1",NaN,NaN,NaN,29694705468.959145,NaN,NaN
228,1887,198506,198606,Drought,NaN,CHN,CHN.11_1,False,states,"CHN.16_1, CHN.31_1, CHN.4_1, CHN.6_1",NaN,NaN,NaN,26508026648.861954,NaN,NaN
229,1887,198506,198606,Drought,NaN,CHN,CHN.12_1,False,states,"CHN.16_1, CHN.31_1, CHN.4_1, CHN.6_1",NaN,NaN,NaN,32386812218.243076,NaN,NaN
230,1887,198506,198606,Drought,NaN,CHN,CHN.13_1,False,states,"CHN.16_1, CHN.31_1, CHN.4_1, CHN.6_1",NaN,NaN,NaN,27990911411.880833,NaN,NaN
231,1887,198506,198606,Drought,NaN,CHN,CHN.14_1,False,states,"CHN.16_1, CHN.31_1, CHN.4_1, CHN.6_1",NaN,NaN,NaN,25785143772.457664,NaN,NaN
232,1887,198506,198606,Drought,NaN,CHN,CHN.15_1,False,states,"CHN.16_1, CHN.31_1, CHN.4_1, CHN.6_1",NaN,NaN,NaN,48178197925.443909,NaN,NaN
233,1887,198506,198606,Drought,NaN,CHN,CHN.16_1,True,states,"CHN.16_1, CHN.31_1, CHN.4_1, CHN.6_1",NaN,NaN,NaN,14848317770.749256,0.141981,NaN
234,1887,198506,198606,Drought,NaN,CHN,CHN.17_1,False,states,"CHN.16_1, CHN.31_1, CHN.4_1, CHN.6_1",NaN,NaN,NaN,14913639043.275215,NaN,NaN
235,1887,198506,198606,Drought,NaN,CHN,CHN.18_1,False,states,"CHN.16_1, CHN.31_1, CHN.4_1, CHN.6_1",NaN,NaN,NaN,37851004211.071609,NaN,NaN


In [180]:
yrmons = pd.Series([int(f"{yr}{str(mon).zfill(2)}") for yr in yrs for mon in range(1, 12+1)], dtype=int)
regions = sorted(emdat_new.state.unique())
dams_ts = pd.DataFrame(0.0, index=regions, columns=yrs)
for row in emdat_new.index:
        
    data_act = emdat_new.loc[row, :]
    # damages distributed to years by equally weighting with event months.
    # put the damage div. number of event months into a pd.Series with the event yrmons as index.
    ev_dams = pd.Series(index=yrmons[(yrmons>=int(data_act.yrmon_begin)) & (yrmons<=int(data_act.yrmon_end))], dtype=float)
    ev_dams[:] = data_act["total_gdp_damage"]/len(ev_dams)
    for yr in sorted(set([str(xx)[:4] for xx in ev_dams.index])): # each event year, as string.
        print(yr)
        dams_ts.loc[data_act.state, int(yr)] = dams_ts.loc[data_act.state, int(yr)] + \
            ev_dams.loc[[xx for xx in ev_dams.index if str(xx)[:4] == yr]].sum()

1980
1980
1980
1980
1983
1984
1985
1980
1981
1982
1983
1984
1985
1983
1982
1983
1984
1985
1983
1984
1985
1981
1982
1983
1980
1981
1982
1983
1984
1985
1982
1983
1984
1985
1983
1980
1981
1982
1983
1984
1985
1980
1981
1982
1983
1984
1985
1980
1981
1982
1983
1984
1985
1982
1983
1984
1985
1981
1982
1983
1984
1985
1983
1980
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1980
1981
1981
1981
1982
1983
1984
1985
1981
1982
1983
1983
1984
1985
1981
1981
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1981
1982
1983
1981
1982
1983
1981
1982
1983
1984
1985
1986
1987
1982
1983
1984
1983
1982
1983
1982
1982
1983
1984
1982
1982
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983


In [181]:
dams_ts

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
AFG,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,461674.463036,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0
AGO,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0
ALB,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0
ARG,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,5.508063e+06,0.000000,0.0
ARM,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0
YUG,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0
ZAF,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,226434.333579,226434.333579,1.483560e+06,1.041909e+06,272843.695364,0.0
ZMB,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.0


In [182]:
yrmons = pd.Series([int(f"{yr}{str(mon).zfill(2)}") for yr in yrs for mon in range(1, 12+1)], dtype=int)
regions = sorted(emdat_new.state.unique())
dams_ts_bin = pd.DataFrame(0.0, index=regions, columns=yrs)
for row in emdat_new.index:
        
    data_act = emdat_new.loc[row, :]
    # damages distributed to years by equally weighting with event months.
    # put the damage div. number of event months into a pd.Series with the event yrmons as index.
    yr_begin=int(data_act.yrmon_begin[:-2])
    yr_end=int(data_act.yrmon_end[:-2])
    ev_dams =np.arange(yr_begin, yr_end+1)
    for yr in ev_dams: # each event year, as string.
        print(yr)
        dams_ts_bin.loc[data_act.state, int(yr)] = 1

1980
1980
1980
1980
1983
1984
1985
1980
1981
1982
1983
1984
1985
1983
1982
1983
1984
1985
1983
1984
1985
1981
1982
1983
1980
1981
1982
1983
1984
1985
1982
1983
1984
1985
1983
1980
1981
1982
1983
1984
1985
1980
1981
1982
1983
1984
1985
1980
1981
1982
1983
1984
1985
1982
1983
1984
1985
1981
1982
1983
1984
1985
1983
1980
1980
1981
1982
1983
1980
1981
1982
1983
1980
1981
1980
1981
1981
1981
1982
1983
1984
1985
1981
1982
1983
1983
1984
1985
1981
1981
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1982
1981
1981
1982
1983
1981
1982
1983
1981
1982
1983
1984
1985
1986
1987
1982
1983
1984
1983
1982
1983
1982
1982
1983
1984
1982
1982
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983
1982
1983


In [150]:
dams_ts_bin

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
AFG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN
AGO,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN
ALB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
ARG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
ARM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WSM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
YUG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
ZAF,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN
ZMB,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN


In [183]:
dams_ts.to_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/damage_distributed/emdat_annual_converted_distributed.csv')

In [184]:
dams_ts_bin.to_csv('/home/insauer/projects/droughts/droughts/data/socio_economic/damage_distributed/emdat_annual_converted_distributed_binary.csv')